# Chapter 1. The World's Smallest Django Project

---

# TOC

## 1. Hello Django

### 1) Creating the View

### 2) The URL Patterns

### 3) The Settings

### 4) Running the Example

## 2. Improvements

### 1) WSGI Application

### 2) Additional Configuration

### 3) Reusable Template

---

## 1. Hello Django

This lightweight “Hello World” project will create a simple Django application using a single-file approach.

### 1) Creating the View

### 2) The URL Patterns

### 3) The Settings

### 4) Running the Example

---

## 2. Improvements

### 1) WSGI Application

### 2) Additional Configuration

### 3) Reusable Template